In [150]:
from modsim import *

http://www.pveducation.org/pvcdrom/2-properties-sunlight/solar-radiation-space

In [151]:
kg= UNITS.kg
m= UNITS.m
N= UNITS.N
W= UNITS.W
K= UNITS.K
j= UNITS.joules
s= UNITS.seconds
degrees= UNITS.degrees


In [152]:
mass_sun= 1.989e30 *kg 
G= 6.673e-11* N *m**2 / kg**2
mass_toast= .05*kg
S_B_const= 5.67e-8 * W/m**2/K**4
temp_S= 5772 *K
area_t= .01 *m 
r_s= 695770 *1000*m 
Pow_s= 3.9e26 * W 
Q_b= .008/273 * j / (kg * K)

power_sun = S_B_const * temp_S *4 *pi *r_s**2 

 

num_G = G* mass_sun *mass_toast   

num_G


 

#I think these are all the equations and constants we need. I found a specific heat for bread that was in kg and C, 
#which I think I converted into Kelvin but it's a bit sketchy. I'm also not sure about the area of the sunward facing 
#toast side, but we can look back at that. 

<Quantity(6.6362985e+18, 'meter ** 2 * newton')>

In [153]:

condition= Condition(x0= 1e30*m, y0= 0*m, velocity= 500*m/s, angle = 40*degrees, duration= 10000*s, T0= 0*K )

In [161]:
#I'm not sure I'm doing this right, I'm mostly copying this from chapter 10 with the intro to vectors and the baseball. 

def make_system(condition):
    unpack(condition)
    
    theta = np.deg2rad(angle)
    
    vx, vy = pol2cart(theta, velocity)
    
    init= State(x=x0, y=y0, vx= vx, vy= vy)
    area_toast = area_t 
    
    ts= linspace(0, duration, 101)
    
    return System(init= init, mass= mass_toast, area_t= area_t, T= T0, Q_b= Q_b, ts= ts)

system= make_system(condition)



In [176]:
def slope_func(state, t, system):
    x, y, vx, vy = state
    
    unpack(system)
    P= Vector(x, y) *m
    V= Vector(vx, vy) *m/s
    Tangent_dir= Vector(x, -y) 
    w= V.proj(Tangent_dir) * Tangent_dir.hat() * m/s 

    F_grav= num_G / P.mag **2 * (P/P.mag) 
    F_cent= w**2  *mass_toast / P.mag  * Tangent_dir.hat()
    
    grav_theta= P.angle
    
    a_grav= (F_grav/mass_toast) * P.hat() *m**2
    a_cent= (F_cent/mass_toast /m) *Tangent_dir.hat() *m**2
    
    
    a1= a_grav + a_cent
    
    a= Vector(a1[0], a1[1])

    
    
    rad_on_toast= t * Pow_s * r_s**2/ P.mag**2 
    temp_toast= rad_on_toast/ Q_b / mass_toast 
    
    
    return vx, vy, a.x, a.y
    
    
    
slope_func(init,1,system)
    

(<Quantity(383.022221559489, 'meter / second')>,
 <Quantity(321.3938048432696, 'meter / second')>,
 <Quantity(1.467060222083663e+35, 'newton / kilogram')>,
 <Quantity(0.0, 'newton / kilogram')>)

In [177]:
run_odeint(system, slope_func)

C:\ProgramData\Miniconda3\lib\site-packages\scipy\integrate\odepack.py:218: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


In [178]:
plot(system.results.)

<bound method NDFrame.head of                      x              y             vx             vy
0.0       1.000000e+30   0.000000e+00   3.830222e+02   3.213938e+02
100.0     1.000000e+30   8.390995e-31   2.841586e+11   3.213938e+02
200.0    7.561849e-312  7.562043e-312  7.562062e-312  7.372954e-315
300.0    2.964394e-323  7.562022e-312  7.562429e-312  7.372954e-315
400.0    2.964394e-323  7.562043e-312  7.562063e-312  7.562023e-312
500.0    7.562043e-312  7.562043e-312  7.562043e-312  7.562043e-312
600.0    7.562043e-312  7.561867e-312  7.561834e-312  7.562480e-312
700.0    7.562025e-312  7.562043e-312  7.562023e-312  7.562043e-312
800.0    7.561834e-312  7.561664e-312  7.562484e-312  7.562485e-312
900.0    7.562485e-312  7.562476e-312  7.562487e-312  7.562487e-312
1000.0   7.562488e-312  7.562488e-312  7.562487e-312  7.562488e-312
1100.0   7.562487e-312  7.562488e-312  7.562488e-312  7.562488e-312
1200.0   7.562487e-312  7.562488e-312  7.562488e-312  7.562484e-312
1300.0   7.562474e

In [165]:
A= Vector(2, -1) *m

B= Vector(0, 5)

A.proj(B)

C= 12*A.hat()


print(type(A), type(A.x))
C.x

<class 'modsim._Vector'> <class 'modsim._Vector'>


<Quantity(10.73312629199899, 'dimensionless')>